In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [8]:
df = pd.read_csv('churn_prediction_cleaned.csv')
df.head()


,age,gender,dependents,occupation,customer_nw_category,current_balance,current_month_credit,previous_month_credit,current_month_debit,previous_month_debit,current_month_balance,previous_month_balance,churn
0,66,Male,0.0,self_employed,2,1458.71,0.20,0.20,0.20,0.20,1458.71,1458.71,0
1,35,Male,0.0,self_employed,2,5390.37,0.56,0.56,5486.27,100.56,6496.78,8787.61,0
2,31,Male,0.0,salaried,2,3913.16,0.61,0.61,6046.73,259.23,5006.28,5070.14,0
3,90,NaN,NaN,self_employed,2,2291.91,0.47,0.47,0.47,2143.33,2291.91,1669.79,1
4,42,Male,2.0,self_employed,3,927.72,0.33,714.61,588.62,1538.06,1157.15,1677.16,1


In [10]:
df_clean = df.copy()

In [14]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28382 entries, 0 to 28381
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   age                     28382 non-null  int64  
 1   gender                  27857 non-null  object 
 2   dependents              25919 non-null  float64
 3   occupation              28302 non-null  object 
 4   customer_nw_category    28382 non-null  int64  
 5   current_balance         28382 non-null  float64
 6   current_month_credit    28382 non-null  float64
 7   previous_month_credit   28382 non-null  float64
 8   current_month_debit     28382 non-null  float64
 9   previous_month_debit    28382 non-null  float64
 10  current_month_balance   28382 non-null  float64
 11  previous_month_balance  28382 non-null  float64
 12  churn                   28382 non-null  int64  
dtypes: float64(8), int64(3), object(2)
memory usage: 2.8+ MB


In [11]:
df_clean.isnull().sum()

age                          0
gender                     525
dependents                2463
occupation                  80
customer_nw_category         0
current_balance              0
current_month_credit         0
previous_month_credit        0
current_month_debit          0
previous_month_debit         0
current_month_balance        0
previous_month_balance       0
churn                        0
dtype: int64

Tratamiento de missings

In [18]:
df_clean["gender"].fillna(method='bfill', inplace=True)

In [21]:
df_dependents_mean = df_clean["dependents"].mean()
df_clean["dependents"].fillna(df_dependents_mean, inplace=True)

In [24]:
df_clean["occupation"].fillna(method='bfill', inplace=True)

Tratamiento de variables categóricas

In [26]:
df_clean["gender"].value_counts()

gender
Male      16848
Female    11534
Name: count, dtype: int64

In [27]:
df_clean["occupation"].value_counts()

occupation
self_employed    17521
salaried          6728
student           2062
retired           2031
company             40
Name: count, dtype: int64